In [308]:
pip install PublicDataReader --upgrade

In [309]:
import PublicDataReader as pdr
import pandas as pd

In [310]:
noin_data = pd.read_csv("../data/native_admdong_sum_2023061.csv")
noin_data["admdong_cd"] =noin_data["admdong_cd"]*100

In [311]:
busstop = pd.read_csv("../data/busstop_address.csv")
busstop_total_total = pd.read_csv("../data/busstop_weight.csv")
busstop["읍면동인구수"] = 0

In [312]:
연결정보 = pdr.code_hdong_bdong()
연결정보 = 연결정보[연결정보["시도명"] == '경상남도']
연결정보 = 연결정보[연결정보["말소일자"].apply(lambda x: len(x) == 0)]
연결정보 = 연결정보[연결정보["읍면동명"].apply(lambda x: len(x) > 0)]
for i in list_data:
    연결정보 = 연결정보[~연결정보["동리명"].str.contains(i)]
연결정보  

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.2.1. 시행)


,시도코드,시도명,시군구코드,시군구명,행정동코드,읍면동명,법정동코드,동리명,생성일자,말소일자
45466,48,경상남도,48121,창원시 의창구,4812125000,동읍,4812125000,동읍,20100701,
45467,48,경상남도,48121,창원시 의창구,4812125000,동읍,4812125021,용강리,20100701,
45468,48,경상남도,48121,창원시 의창구,4812125000,동읍,4812125022,용전리,20100701,
45469,48,경상남도,48121,창원시 의창구,4812125000,동읍,4812125023,남산리,20100701,
45470,48,경상남도,48121,창원시 의창구,4812125000,동읍,4812125024,덕천리,20100701,
...,...,...,...,...,...,...,...,...,...,...
51144,48,경상남도,48890,합천군,4889046000,용주면,4889046032,고품리,19880423,
51145,48,경상남도,48890,합천군,4889046000,용주면,4889046033,봉기리,19880423,
51147,48,경상남도,48890,합천군,4889046000,용주면,4889046035,우곡리,19880423,
51148,48,경상남도,48890,합천군,4889046000,용주면,4889046036,죽죽리,19880423,


### 주소를 법정동 코드 -> 행정동 코드로 변환 1번 방법



In [313]:
p = 0
for q in range(len(busstop["도시코드"].unique())):
    연결정보_set = 연결정보[연결정보["시군구코드"] == q]
    busstop_set = busstop[busstop["도시코드"] == q]
    for i in range(len(busstop_set["상세주소"])):
        for j in range(len(연결정보_set["동리명"])):
            if 연결정보_set["동리명"][j] in busstop_set["상세주소"][i]:
                for k in range(len(noin_data["admdong_cd"])):
                    if str(noin_data["admdong_cd"][k]) in str(연결정보_set["행정동코드"][j]):
                        busstop["읍명동인구수"][i + p] = noin_data["0"][k]
                        break
                break
    p += len(busstop_set["상세주소"])


### 2번 방법

In [314]:
연결정보['행정동코드'] = 연결정보['행정동코드'].astype(str)
noin_data['admdong_cd'] = noin_data['admdong_cd'].astype(str)
연결정보 = 연결정보.merge(noin_data, left_on='행정동코드', right_on='admdong_cd', how='left')
list_data = ["창원시", "거제시", "김해시", "양산시", "진주시", "함양군", "거창군", "합천군", "사천시", "하동군", "밀양시", "통영시", "마산", "함안군", "창녕군", "진해", "의령군","남해군", "고성군"]

# 각 도시코드에 대해 처리
for city_code in list_data:
    busstop_set = busstop[busstop["도시명"].str.contains(city_code)]
    연결정보_set = 연결정보[연결정보["시군구명"].str.contains(city_code)]
    for i, bus_row in busstop_set.iterrows():
        상세주소 = bus_row['상세주소']
        for j, conn_row in 연결정보_set.iterrows():
            동리명 = conn_row['동리명']
            if 동리명 in 상세주소:
                행정동코드 = str(conn_row['행정동코드'])
                matching_noin = noin_data[noin_data['admdong_cd'].astype(str) == 행정동코드]
                if not matching_noin.empty:
                    busstop.at[i, '읍면동인구수'] = matching_noin['0'].values[0]
                    break

In [315]:
busstop_zero = busstop[busstop["읍면동인구수"] == 0]
for city_code in list_data:
    busstop_set = busstop_zero[busstop_zero["상세주소"].str.contains(city_code)]
    연결정보_set = 연결정보[연결정보["시군구명"].str.contains(city_code)]
    for i, bus_row in busstop_set.iterrows():
        상세주소 = bus_row['상세주소']
        for j, conn_row in 연결정보_set.iterrows():
            동리명 = conn_row['동리명']
            if 동리명 in 상세주소:
                행정동코드 = str(conn_row['행정동코드'])
                matching_noin = noin_data[noin_data['admdong_cd'].astype(str) == 행정동코드]
                if not matching_noin.empty:
                    busstop_zero.at[i, '읍면동인구수'] = matching_noin['0'].values[0]
                    break

In [318]:
busstop = busstop.merge(busstop_zero[["정류장번호", "읍면동인구수"]], on = "정류장번호", how = "left")

In [332]:
busstop =busstop[busstop["정류장번호"] != "GHB2089"]
busstop_total_total =busstop_total_total[busstop_total_total["정류장번호"] != "GHB2089"]

In [358]:
busstop = busstop.rename(columns={"읍면동인구수_x" : "읍면동인구수"})
busstop 

,Unnamed: 0,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,geometry,상세주소,읍면동인구수,읍면동인구수_y
0,0,CWB379000566,구암고등학교,35.246296,128.593978,2023-10-16,420615.0,38010,경상남도 창원시,창원,POINT (1099529.3099473445 1695090.3215359906),경상남도 창원시 마산회원구 구암동,410959,NaN
1,1,CWB379000567,사격장입구,35.244923,128.683156,2023-10-16,107508.0,38010,경상남도 창원시,창원,POINT (1107644.8457543373 1695031.1020157929),경상남도 창원시 의창구 사림동,1934447,NaN
2,2,CWB379000568,장애인복지회관입구,35.244960,128.673597,2023-10-16,119117.0,38010,경상남도 창원시,창원,POINT (1106775.0680812967 1695024.859684883),경상남도 창원시 의창구 봉곡동,1934447,NaN
3,3,CWB379000570,장애인복지회관입구,35.244773,128.673243,2023-10-16,119118.0,38010,경상남도 창원시,창원,POINT (1106743.105090053 1695003.6802969584),경상남도 창원시 의창구 봉곡동,1934447,NaN
4,4,CWB379000573,지귀상가,35.244778,128.659437,2023-10-16,119111.0,38010,경상남도 창원시,창원,POINT (1105486.9448440736 1694989.4555102403),경상남도 창원시 의창구 봉곡동,1934447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18856,18854,YSB5166,사과나무집,35.491970,128.989600,2023-10-16,5166.0,38100,경상남도 양산시,양산,POINT (1135114.8887175242 1722807.740568875),경상남도 양산시 원동면 선리,2291970,NaN
18857,18855,YSB5167,이지더원3차,35.322630,129.027700,2023-10-16,5167.0,38100,경상남도 양산시,양산,POINT (1138861.2639738221 1704077.9474447218),경상남도 양산시 동면 석산리,2291970,NaN
18858,18856,YSB5168,반도유보라6차,35.317890,129.027500,2023-10-16,5168.0,38100,경상남도 양산시,양산,POINT (1138851.1887177934 1703551.928831223),경상남도 양산시 동면 석산리,2291970,NaN
18859,18857,YSB5169,양우내안애6차,35.318010,129.023400,2023-10-16,5169.0,38100,경상남도 양산시,양산,POINT (1138478.2608111338 1703559.5004212412),경상남도 양산시 동면 석산리,2291970,NaN


In [365]:
busstop_total_total.drop_duplicates(subset='정류장번호')

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,geometry,weight,읍면동인구수
0,CWB379000566,구암고등학교,35.246296,128.593978,2023-10-16,420615.0,38010,경상남도 창원시,창원,POINT (1099529.3099473445 1695090.3215359906),478654.0,410959
1,CWB379000567,사격장입구,35.244923,128.683156,2023-10-16,107508.0,38010,경상남도 창원시,창원,POINT (1107644.8457543373 1695031.1020157929),464401.0,1934447
2,CWB379000568,장애인복지회관입구,35.244960,128.673597,2023-10-16,119117.0,38010,경상남도 창원시,창원,POINT (1106775.0680812967 1695024.859684883),732161.0,1934447
3,CWB379000570,장애인복지회관입구,35.244773,128.673243,2023-10-16,119118.0,38010,경상남도 창원시,창원,POINT (1106743.105090053 1695003.6802969584),266638.0,1934447
4,CWB379000573,지귀상가,35.244778,128.659437,2023-10-16,119111.0,38010,경상남도 창원시,창원,POINT (1105486.9448440736 1694989.4555102403),1120637.0,1934447
...,...,...,...,...,...,...,...,...,...,...,...,...
19023,YSB5166,사과나무집,35.491970,128.989600,2023-10-16,5166.0,38100,경상남도 양산시,양산,POINT (1135114.8887175242 1722807.740568875),3171.0,2291970
19024,YSB5167,이지더원3차,35.322630,129.027700,2023-10-16,5167.0,38100,경상남도 양산시,양산,POINT (1138861.2639738221 1704077.9474447218),147088.0,2291970
19025,YSB5168,반도유보라6차,35.317890,129.027500,2023-10-16,5168.0,38100,경상남도 양산시,양산,POINT (1138851.1887177934 1703551.928831223),147088.0,2291970
19026,YSB5169,양우내안애6차,35.318010,129.023400,2023-10-16,5169.0,38100,경상남도 양산시,양산,POINT (1138478.2608111338 1703559.5004212412),147088.0,2291970


### 병합

In [350]:
# busstop_total_total = busstop_total_total.rename(columns={"nodeid" : "정류장번호"})
busstop_total_total = busstop_total_total.merge(busstop[["정류장번호", "읍면동인구수"]], on = "정류장번호", how='left')

In [368]:
busstop_total_total.to_csv("../data/bustop_pop.csv", index = False)